## Global Giving API

Global Giving Foundation is a non-profit that vets other non-profits and charities. The response only returns 10 results for each request.  While there are more entries, the "hasNext" value will be true.

https://www.globalgiving.org/api

In [5]:
import boto3
import requests
import os

from configparser import ConfigParser
from collections import OrderedDict
from retrying import retry
from io import StringIO

ini_file = os.path.join(os.pardir, "config", "config.ini")

def get_ini_vals(ini_file, section):
    config = ConfigParser()
    config.read(ini_file)
    return config[section]
    
country = 'US'
keys = get_ini_vals(ini_file, "globalgiving.api")
print("Reading configs from from %s" % ini_file)
print(keys)

Reading configs from from ../config/config.ini
<Section: globalgiving.api>


### Projects API

Tags of interest:  
```
<project>
    <hasNext>true</hasNext>   # if there are more results
    <activities>Energy healers have been researching the effects of metals and the elements on the health of the human body. CHI plans to hold workshops to train energy healers to utilize various metals and the elements in promoting healthier lifestyles.
    </activities>
```
(address/time info)

```
        <approvedDate>2003-05-20T19:40:05-04:00</approvedDate>
        <contactAddress>Creating Hope International</contactAddress>
        <contactAddress2>P. O. Box 1058</contactAddress2>
        <contactCity>Dearborn</contactCity>
        <contactCountry>United States</contactCountry>
        <contactPostal>48121</contactPostal>
        <contactState>Michigan</contactState>
        <contactUrl>http://www.creatinghope.org</contactUrl>
        <country>United States</country>
        
        <image>
            <imagelink size="medium">
                <url>oaiwefjslc.jpg</url>
            </imagelink>
        </image>
```
(organization-images)

```
        <logoUrl>https://dpqe0zkrjo0ak.cloudfront.net/pfil/organ/254/orglogo.jpg</logoUrl>

```

In [6]:
url= "https://api.globalgiving.org/api/public/projectservice/countries/{country_}/projects".format(country_=country)
session = requests.session()
response = session.get(url, params={'api_key':'37466a9c-154d-4d82-aff8-7cb02a1bfbb1'})

In [7]:
response_text = response.text
print(response.text)

<?xml version="1.0" encoding="UTF-8" standalone="yes"?>
<projects numberFound="2428">
    <hasNext>true</hasNext>
    <nextProjectId>607</nextProjectId>
    <project>
        <active>false</active>
        <activities>Energy healers have been researching the effects of metals and the elements on the health of the human body. CHI plans to hold workshops to train energy healers to utilize various metals and the elements in promoting healthier lifestyles.</activities>
        <additionalDocumentation>https://www.globalgiving.org/pfil/16/projdoc.doc</additionalDocumentation>
        <approvedDate>2003-05-20T19:40:05-04:00</approvedDate>
        <contactAddress>Creating Hope International</contactAddress>
        <contactAddress2>P. O. Box 1058</contactAddress2>
        <contactCity>Dearborn</contactCity>
        <contactCountry>United States</contactCountry>
        <contactPostal>48121</contactPostal>
        <contactState>Michigan</contactState>
        <contactUrl>http://www.creatinghop

In [41]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(response_text, 'html.parser')
print(soup.prettify())

<?xml version="1.0" encoding="UTF-8" standalone="yes"?>
<projects numberfound="2424">
 <hasnext>
  true
 </hasnext>
 <nextprojectid>
  607
 </nextprojectid>
 <project>
  <active>
   false
  </active>
  <activities>
   Energy healers have been researching the effects of metals and the elements on the health of the human body. CHI plans to hold workshops to train energy healers to utilize various metals and the elements in promoting healthier lifestyles.
  </activities>
  <additionaldocumentation>
   https://www.globalgiving.org/pfil/16/projdoc.doc
  </additionaldocumentation>
  <approveddate>
   2003-05-20T19:40:05-04:00
  </approveddate>
  <contactaddress>
   Creating Hope International
  </contactaddress>
  <contactaddress2>
   P. O. Box 1058
  </contactaddress2>
  <contactcity>
   Dearborn
  </contactcity>
  <contactcountry>
   United States
  </contactcountry>
  <contactpostal>
   48121
  </contactpostal>
  <contactstate>
   Michigan
  </contactstate>
  <contacturl>
   http://www.cr

In [59]:
print(soup.hasnext)
"true" in soup.hasnext
print(soup.nextprojectid.contents)

<hasnext>true</hasnext>
['607']


In [60]:
projects = soup.find_all('project')


In [86]:
project1 = projects[0]
print(project1.theme.id.string)

children


In [ ]:
# parse project
project_keys = [
    'action_tags',
    'issue_tags',
    'org_name',
    'org_url',
    'org_street',
    'org_city',
    'org_state',
    'org_zip',
    'org_logo',
    'project_name',
    'project_date',
    'project_img',
    'project_url',
    
]



def parse_project(project):
    

In [1]:
def print_all_results(response_soup):
    """get all project info
    """
    if response_soup.hasnext.string == "true":
        next_id = response_soup.nextprojectid.string
        projects = response_soup.find_all('project')
        for project in projects:
            if project.active.contents[0]=="true":
                print(project.title.contents, 
                      project.id.contents,
                      project.find_all('theme').name
                     )
        
        request_args.update({'nextProjectId': next_id})
        response = session.get(url, params=request_args)
        soup = BeautifulSoup(response.text, 'html.parser')
        return print_all_results(soup)
    
    else:
        projects = response_soup.find_all('project')
        for project in projects:
            print(project.title.contents, project.id.contents)
            
        return None
        
print_all_results(soup)
        

NameError: name 'soup' is not defined

In [13]:
# API Class
country = "US"
base_url = "api.globalgiving.org/api/public"
end_point = "/projectservice/countries/{country_}/projects".format(country_=country)
request_args = {'api_key': keys['api_key']}

class GlobalGivingException(Exception):
    pass

class GlobalGivingAPI(object):
    def __init__(self, 
                 host, 
                 endpoint, 
                 params,
                 protocol="https", 
                 verify=False):
        self.protocol = protocol
        self.host = host
        self.endpoint = endpoint
        self.session = requests.session()
        self.session.verify = verify
        self.params = params
        
    
    @retry(stop_max_attempt_number=10, wait_fixed=500)
    def _get(self):
        url = self.protocol + '://' + self.host + self.endpoint

        try:
            response = self.session.get(url, params=self.params)
            if 'error_message' in response:
                raise GlobalGivingException(response['error_message'])
        except (requests.RequestException, ValueError) as e:
            print(url, kwargs)
            response = None

        return response
    
    def _gg_has_next(xml_soup):
        
    
    def _gg_xml_iter(self):
        """Iterate over api responses until no more hasNext tags.
        """
        has_next = True
        while self._gg_has_next():
            
    
    def get_gg_results(self):
        results = self._get().text
        results_soup = BeautifulSoup(results, 'html.parser')
        for result in results.text:
            yield result


In [12]:
gg_api = GlobalGivingAPI(base_url, end_point, request_args)

for i in gg_api.response_generator():
    print(i.text)

/Users/safyre/anaconda/envs/imwithdata/lib/python3.5/site-packages/requests/packages/urllib3/connectionpool.py:852: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


AttributeError: 'bytes' object has no attribute 'text'

# ElasticSearch

https://elasticsearch-py.readthedocs.io/en/master/